<a href="https://colab.research.google.com/github/bhanuchaddha/The-Ai-Handbook/blob/main/4-AI-Agents/retail_arrangement_analysis_agent/Multi_Agent_Retail_Arrangement_Analysis_Tool_with_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retail Arrangement Analysis Tool

## What Will We Create?
We will create a **Retail Arrangement Analysis Tool**, an innovative AI-powered solution designed to analyze images of store shelves and provide detailed insights into product arrangements. This tool will identify store types, evaluate goods placement, and suggest actionable recommendations to optimize store layout and boost sales. By leveraging AI, computer vision, and retail market expertise, this tool will serve as the ultimate assistant for store managers and market analysts striving for retail excellence.

In short, we’re bringing Sherlock Holmes’s attention to detail to the world of retail, minus the Victorian hat.

---

## Backstory

Meet Michal, the proud owner of a bustling **clothing store**. Always striving for excellence, Michal dreams of creating a seamless shopping experience for customers while maximizing sales potential. However, managing shelf arrangements and keeping up with market trends is no walk in the park. That’s where the **Goods Arrangement Analysis Tool** steps in to save the day.

One day, Michal notices that the store’s displays look more like a rummage sale than a curated fashion haven. With the help of this tool, Michal uploads images of the store’s sections, receives a detailed breakdown of the arrangement, and gets expert recommendations from the Market Trend Advisor agent.

With a little AI magic, Michal can now:

- Spot issues like cluttered racks or underutilized display space.
- Understand how to reorganize goods for better visibility and appeal.
- Gain market-aligned insights to enhance customer satisfaction and boost sales.

Retail harmony achieved, Michal can finally focus on expanding the business and maybe even plan that long-overdue vacation.

---

## Technology Being Used
Our project leverages cutting-edge technology and robust libraries to deliver insights with precision:

- **Python**: The backbone of our application, tying everything together like a seasoned maestro.
- **CrewAI**: A powerful multi-agent framework that enables seamless collaboration between roles like Store Planner and Market Analyst.
- **OpenAI GPT-4o-mini**: A lightweight but capable LLM for generating detailed and insightful analyses.
- **Gradio**: A simple yet effective library to create a user-friendly web interface for image uploads and result viewing.
- **DuckDuckGo Search API**: For sourcing additional market trends and data, because Google’s not the only search wizard in town.

---

## Implementation


### Code
The codebase is structured into modular components:
- **Image Upload Functionality**: Uploads images to catbox.moe and retrieves the image URL.
- **Goods Arrangement Analysis Tool**: Processes the image using GPT-4o-mini to identify store types and analyze arrangements.
- **DuckDuckGo Search Tool**: Provides market insights to align recommendations with current trends.
- **Agents and Tasks**: Simulate a collaborative workflow between the Store Manager and Market Analyst.
- **Main Crew Process**: Orchestrates the entire process, ensuring smooth execution of tasks.

### Frontend Using Gradio
To make the application user-friendly, we’ll integrate **Gradio** for the frontend. This web interface will allow users to:
1. Upload images of store shelves.
2. View detailed analysis reports.
3. Access actionable recommendations.

Gradio provides a clean, drag-and-drop interface that even your grandma could use (though she might mistake it for a virtual puzzle game).

---

## Agents
Agents are the heart of the system, each designed with specific roles and expertise to ensure seamless collaboration:

1. **Store Manager Agent**

   - **Role**: Store Arrangement Planner.
   - **Goal**: Analyze the arrangement of goods in the store and generate a detailed report for the Market Analyst.
   - **Backstory**: This agent specializes in examining store layouts, assessing product placement, and optimizing space utilization.
   - **Tool Used**: Goods Arrangement Analysis Tool for identifying store types and analyzing goods arrangements.

2. **Market Analyst Agent**

   - **Role**: Market Trend Advisor.
   - **Goal**: Provide recommendations for improving goods arrangements based on market trends.
   - **Backstory**: This agent possesses deep knowledge of market trends and consumer behavior, enabling effective recommendations for enhanced sales and customer satisfaction.
   - **Tool Used**: DuckDuckGo Search Tool for gathering market insights.

---

## Tasks
Tasks define the workflow and guide agents in achieving their goals. Here’s the breakdown:

1. **Analyze Arrangement**

   - **Description**: Use the Goods Arrangement Analysis Tool to identify the store type and analyze the current arrangement of goods in the provided image.
   - **Output**: A detailed report including the store type, product placement, and any visible issues.
   - **Assigned Agent**: Store Manager Agent.

2. **Get Recommendations**

   - **Description**: Review the arrangement report and use market knowledge to generate actionable recommendations for optimization.
   - **Output**: A set of recommendations aligned with market trends and consumer preferences.
   - **Assigned Agent**: Market Analyst Agent.

3. **Create Arrangement Plan**

   - **Description**: Develop a detailed plan for rearranging goods based on the Market Analyst’s recommendations.
   - **Output**: A comprehensive action plan for improving goods arrangement.
   - **Assigned Agent**: Store Manager Agent.

---

## Crew
The Crew component brings it all together, orchestrating the interaction between agents and tasks:

- **Agents Involved**: Store Manager Agent and Market Analyst Agent.
- **Tasks Sequence**: Tasks are executed in a sequential process to ensure logical flow and completeness.
- **Process Management**: CrewAI’s `Crew` and `Process` classes manage task execution, ensuring that each agent fulfills its role effectively.
- **Collaboration**: Agents collaborate seamlessly to produce actionable insights and practical plans for store optimization.

---

## How to Use the App

Using the **Goods Arrangement Analysis Tool** is as simple as shopping for your favorite snacks (which, incidentally, you’ll find better organized thanks to this app):

1. **Upload an Image**: Drag and drop an image of your store’s shelf or section into the Gradio interface.
2. **Analyze Goods Arrangement**:
   - The tool identifies the store type (e.g., grocery aisle, electronics display).
   - It provides a detailed breakdown of the goods arrangement, including product placement, visible issues, and space utilization.
3. **Get Recommendations**: The Market Analyst agent evaluates the report and generates actionable suggestions aligned with current trends.
4. **Download the Plan**: The Store Planner agent transforms recommendations into a practical rearrangement plan, which you can download and implement.

Now you’re equipped to turn your store into a retail paradise. As a bonus, your customers might even think you hired a retail wizard.

---

By combining AI, retail expertise, and a touch of humor, the **Goods Arrangement Analysis Tool** aims to revolutionize the way stores optimize their layouts. It’s not just a tool; it’s your new retail superhero.



In [ ]:
!pip install crewai langchain openai gradio httpx python-dotenv  duckduckgo-search langchain-community

In [2]:
from google.colab import userdata
import os
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

# Setup

In [ ]:
import base64
import gradio as gr
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata, files
import os
from io import BytesIO
from typing import Optional
from PIL import Image
from openai import OpenAI
import requests  # Import the requests library
import logging  # Import the logging module

# Basic logging configuration - try this first
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

client = OpenAI()

# Retrieve API Key from Colab Secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

# Load environment variables
load_dotenv()

# Set up language model
llm = ChatOpenAI(model="gpt-4o-mini")  # Or another suitable model

In [71]:


def upload_image_to_catbox(image_path):
    """Uploads an image to catbox.moe and returns the URL."""
    url = 'https://catbox.moe/user/api.php'
    data = {'reqtype': 'fileupload', 'userhash': ''}
    try:
        files = {'fileToUpload': open(image_path, 'rb')}
        response = requests.post(url, data=data, files=files)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.text.strip()
    except Exception as e:
        return f"Error uploading image: {e}"

class GoodsArrangementAnalysisTool(BaseTool):
    name: str = "Goods Arrangement Analysis Tool"
    description: Optional[str] = "Useful for getting a detailed description of the existing arrangement of goods in an image, identifying the store type, product names, placement, and any issues. Input should be an image URL."

    def _run(self, image_url: str) -> str:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": """Analyze this image to identify the type of store or section it is (e.g., grocery aisle, clothing rack, electronics display, etc.) and describe the current arrangement of goods in detail.
                                Pay close attention to product names, product placement, and any visible issues.
                                Please provide the store type first, followed by the detailed description of the goods arrangement.""",
                            },
                            {
                                "type": "image_url",
                                "image_url": {"url": image_url},
                            },
                        ],
                    }
                ],
                max_tokens=1024,
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error analyzing goods arrangement: {e}"

class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGo Search"
    description: Optional[str] = "A tool for searching the web using DuckDuckGo"

    def _run(self, query: str) -> str:
        search = DuckDuckGoSearchAPIWrapper()
        return search.run(query)

# Define agents
store_manager = Agent(
    role='Store Arrangement Planner',
    goal='Analyze the arrangement of goods in the physical store and provide a detailed report to the market analyst for recommendations.',
    backstory="""As the Store Arrangement Planner, you are responsible for examining the store's layout and the arrangement of goods.
    You assess product placement, and optimize space utilization across different store types or sections.
    You have access to advanced tools for visualizing goods arrangement, which help you
    collect accurate data on the current placement of products.
    You are capable to translate market analysis into a plan for the store
    or specific sections.""",
    verbose=True,
    tools=[GoodsArrangementAnalysisTool()],
    llm=llm,
    allow_delegation=False
)

market_analyst = Agent(
    role='Market Trend Advisor',
    goal='Provide recommendations to improve the arrangement of goods based on market trends.',
    backstory="""As the Market Trend Advisor, you possess in-depth knowledge of market trends and consumer behavior.
    Your experience and keen sense of retail enable you to propose effective recommendations
    for different store types and sections. You analyze reports provided by the Store Arrangement Planner to suggest
    improvements that enhance sales and customer experience.""",
    verbose=True,
    tools=[DuckDuckGoSearchTool()],
    llm=llm,
    allow_delegation=True
)

# Define tasks
def create_tasks(image_url):
    analyze_arrangement = Task(
        description=f"""Use the Goods Arrangement Analysis Tool to identify the store type and collect visual data on the current arrangement of goods from the provided image URL.
        The image URL is: '{image_url}'.
        Prepare a detailed report highlighting the current layout, products, product placement, and any observed issues.
        Ensure the report is detailed at the level of product names and begins with the identified store type.""",
        expected_output="""A comprehensive report on the identified store type, including the store type name, visual data,
        analysis of the goods arrangement, space utilization, and any initial observations.""",
        agent=store_manager
    )

    get_recommendations = Task(
        description="""Review the report on the goods arrangement provided by the Store Arrangement Planner.
        Utilize your immense knowledge of the retail market and internet to assess current trends relevant to the products in this store type.
        Develop expert recommendations to optimize sales and customer satisfaction through improved goods arrangement.
        Ensure the recommendations are detailed and include specific product names or categories.""",
        expected_output="""A set of actionable recommendations for improving the arrangement of goods in the store type,
        aligned with current market trends and consumer preferences.""",
        agent=market_analyst
    )

    create_arrangement_plan = Task(
        description=f"""List the recommendations from the Market Trend Advisor.
        Then develop a detailed plan for the Store Arrangement Planner and relevant staff to implement the recommended changes to the goods arrangement.
        The image URL is: '{image_url}'.
        Ensure the plan includes the market analyst recommendations.
        Ensure the plan is practical and outlines the steps needed to rearrange the existing goods effectively.
        Ensure to specify which new products, if any, should be added or where existing ones should be moved.
        Be smart and well explained.
        Give the explanation of your recommendations and the goal to achieve.""",
        expected_output="""A detailed list of recommendations and an action plan for improving the arrangement of goods according to market trends,
        including market analyst recommendations and translation into practical tasks for the Store Arrangement Planner and relevant staff.
        Include the following sections in the markdown output:

        1. **Changes to Existing Arrangements:** Specific steps for rearranging current products.
        2. **Suggestions for New Items or Relocations:** Which new products should be added or where existing ones should be moved.
        3. **Other Suggestions for Store Performance:** Additional recommendations based on market analysis.
        """,
        agent=store_manager
    )
    return [analyze_arrangement, get_recommendations, create_arrangement_plan]

# Define crew creation function
def create_crew(tasks):
    retail_crew = Crew(
        agents=[store_manager, market_analyst],
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )
    return retail_crew

# Define main function to run CrewAI process
def run_crewai(image_url):
    try:
        tasks = create_tasks(image_url)
        crew = create_crew(tasks)
        result = crew.kickoff()
        return result
    except Exception as e:
        return f"Error processing image or running CrewAI: {e}"



In [ ]:
# --- Direct Colab Execution ---
# if __name__ == "__main__":
#     print("Please upload an image file:")
#     uploaded = files.upload()

#     if uploaded:
#         image_filename = list(uploaded.keys())[0]
#         # Save the uploaded image to a temporary file
#         with open(image_filename, 'wb') as f:
#             f.write(uploaded[image_filename])

#         # Upload the image to Catbox
#         image_url = upload_image_to_catbox(image_filename)

#         # Clean up the temporary file
#         os.remove(image_filename)

#         if image_url and not image_url.startswith("Error"):
#             print(f"Image uploaded successfully. Catbox URL: {image_url}")
#             output = run_crewai(image_url)
#             print("\n--- Analysis and Action Plan ---")
#             print(output)
#         else:
#             print(f"Error uploading image to Catbox: {image_url}")
#     else:
#         print("No image file uploaded.")

In [86]:
# Gradio function to handle image upload and run CrewAI
def analyze_image_and_plan(image):
    if image is None:
        return "No image uploaded."

    try:
        # Save the uploaded image to a temporary file
        image_filename = "temp_image.png"  # You can use a more unique name if needed
        image.save(image_filename)

        # Upload the image to Catbox
        image_url = upload_image_to_catbox(image_filename)

        # Clean up the temporary file
        os.remove(image_filename)

        if image_url and not image_url.startswith("Error"):
            print(f"Image uploaded successfully. Catbox URL: {image_url}")
            output = run_crewai(image_url)
            return str(output)
        else:
            return f"Error uploading image to Catbox: {image_url}"
    except Exception as e:
        return f"Error processing image: {e}"

In [88]:
# Gradio interface
with gr.Blocks() as iface:
    # Include custom CSS in HTML
    gr.HTML(
        """
        <style>
        #main-header {
            text-align: center;
            font-family: 'Arial', sans-serif;
            font-size: 24px;
            color: #2A2A2A;
            margin-bottom: 20px;
        }

        #image-upload label {
            font-size: 16px;
            color: #2A2A2A;
            font-weight: bold;
        }

        #recommend-button {
            background-color: #4CAF50 !important;
            color: white !important;
            border-radius: 5px !important;
            padding: 10px 20px !important;
            margin-top: 10px !important;
            font-size: 16px !important;
            border: none !important;
            cursor: pointer !important;
        }

        #recommend-button:hover {
            background-color: #45A049 !important;
        }

        #output-area {
            font-size: 16px;
            color: #2A2A2A;
            margin-top: 30px;
            border-top: 1px solid #DDD;
            padding-top: 15px;
        }
        </style>
        """
    )

    # Header Section
    gr.Markdown(
        """
        # Retail Arrangement Analysis Tool
        Upload your store layout image to receive AI-driven recommendations for an optimal arrangement.
        """,
        elem_id="main-header"
    )

    # Main Upload Section
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(
                type="pil",
                label="Upload Store Image",
                elem_id="image-upload"
            )
            recommend_button = gr.Button(
                "Get Recommendation",
                variant="secondary",
                elem_id="recommend-button"
            )

    # Output Section
    plan_output = gr.Markdown(
        "### Store Arrangement Plan\nResults will appear here after processing.",
        elem_id="output-area"
    )

    # Button Click Event
    recommend_button.click(
        fn=analyze_image_and_plan,
        inputs=image_input,
        outputs=plan_output
    )

iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dee05ce79f455fca4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Image uploaded successfully. Catbox URL: https://files.catbox.moe/h8koxb.png
# Agent: Store Arrangement Planner
## Task: Use the Goods Arrangement Analysis Tool to identify the store type and collect visual data on the current arrangement of goods from the provided image URL.
        The image URL is: 'https://files.catbox.moe/h8koxb.png'.
        Prepare a detailed report highlighting the current layout, products, product placement, and any observed issues.
        Ensure the report is detailed at the level of product names and begins with the identified store type.


# Agent: Store Arrangement Planner
## Thought: I need to analyze the provided image URL to gather information about the store type, layout, product placement, and any issues present in the arrangement of goods.
## Using tool: Goods Arrangement Analysis Tool
## Tool Input: 
"{\"image_url\": \"https://files.catbox.moe/h8koxb.png\"}"
## Tool Output: 
**Store Type:** Grocery Store (Produce Section)

**Detailed Description:**

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(




# Agent: Market Trend Advisor
## Thought: I need to gather current market trends and consumer preferences related to grocery store produce sections to make informed recommendations for the arrangement of goods.
## Using tool: DuckDuckGo Search
## Tool Input: 
"{\"query\": \"current trends in grocery store produce section arrangement 2023\"}"
## Tool Output: 
For grocery stores, last year held plenty of challenges - from high food-at-home prices to increased competition from non-traditional grocery players like dollar stores and superstores. But 2023 also offered the segment plenty of opportunities. Discount chains made a strong showing - and customers spent more time browsing grocery aisles, loading up on essentials and making every trip to ... Traditional supermarkets and health/organic specialty stores have been losing share since 2020. Produce ecommerce was an area of growth for several years but lost ground in 2023. Food waste's financial impact may reduce produce purchases. Six 